In [46]:
import os
import pandas as pd
import geopandas as gpd
os.chdir(os.path.expanduser('~/TOP-Sprint/analysis'))
from src.features.count_jobs import count_jobs
from src.features.count_jobs import count_all_jobs
from src.features.jobs_vulnerability import get_worker_svi
from src.features.build_stop_features import get_transit_walksheds
from src.features.areal_interpolation import areal_interpolation
from src.features.count_jobs import count_jobs_to_subtract
import src.features.areal_interpolation as areal 
from src.features.build_stop_features import get_stops_features
from src.features.build_stop_features import get_svi
from src.data.process_stops import make_stops

In [ ]:
BASE_WALKSHED_PATH = '/home/data/osm/'
CITY_DIRS = ['nyc','hampton_roads']

NYC_TRANSIT_WALKSHED = '/home/data/osm/nyc/walksheds/transit_walkshed_fixed.geojson'
HR_TRANSIT_WALKSHED = '/home/data/osm/hampton_roads/walksheds/transit_walkshed_fixed.geojson'

NYC_WALK_GRAPH = '/home/data/osm/nyc/NYC_walk_graph.gpickle'
HR_WALK_GRAPH = '/home/data/osm/hampton_roads/HR_walk_graph.gpickle'

NYC_LODES = '/home/data/census/nyc/LODES/ny_od_main_JT01_2019.csv'
HR_LODES = '/home/data/census/hampton_roads/LODES/va_od_main_JT01_2019.csv'

NYC_BLOCKS = '/home/data/census/nyc/geo/block_groups.geojson'
HR_BLOCKS = '/home/data/census/hampton_roads/geo/block_groups.geojson'

NYC_TRACTS = '/home/data/census/nyc/geo/tracts.geojson'
HR_TRACTS = '/home/data/census/hampton_roads/geo/tracts.geojson'

SVI_PATH = '/home/data/social_vulnerability_index/SVI2020_US.csv'

In [ ]:
nyc_stops = ps.process_feeds(["/home/data/transit_feed_data/mta_feeds/"], "nyc")

In [ ]:
nyc_stops.drop("routes_serviced", 1).drop_duplicates()

In [ ]:
stops_raw = geopd.read_file("/home/data/results/GTFS_stops_processed.geojson")

In [ ]:
stops_raw

In [ ]:
nyc_LODES

In [16]:
svi = get_svi()

Getting NYC SVI
prep SVI
prep LODES
prep census
get interpolated jobs
merge SVI
aggregate SVI
Getting Hampton Roads SVI
prep SVI
prep LODES
prep census
get interpolated jobs
merge SVI
aggregate SVI


In [35]:
svi

,stop_id,SVI_total,SVI_SES,SVI_household,SVI_race,SVI_housing_transport,worker_vulnerability_cat
0,404028,9.868632,3.188302,2.828710,0.793351,3.057867,2
1,404270,9.880519,3.193774,2.833208,0.794443,3.058788,2
2,403485,9.879350,3.193190,2.832947,0.794352,3.058555,2
3,405435,9.912111,3.225876,2.826416,0.798160,3.060642,2
4,405436,9.945778,3.240409,2.833493,0.800562,3.070323,2
...,...,...,...,...,...,...,...
2940,5715,7.241358,2.437004,2.360032,0.608692,1.835728,0
2941,2917,7.941593,2.687577,2.556762,0.654092,2.043189,2
2942,2949,7.941593,2.687577,2.556762,0.654092,2.043189,2
2943,0873,7.432529,2.527524,2.397487,0.617620,1.890071,0


In [94]:
stops.columns

Index(['stop_id', 'route_type', 'stop_name', 'stop_lat', 'stop_lon',
       'feed_name', 'routes_serviced', 'routes_serviced_str', 'GEOID_2010',
       'GEOID_2020', 'geometry', 'city'],
      dtype='object')

In [106]:
duplicated_stops = svi.groupby("stop_id").agg(count = ("stop_id", "count")).reset_index().query("count > 1").stop_id
duplicated_stops

635      100025
637      100027
641      100033
644      100039
648      100045
          ...  
17952    904246
17954    904250
17957    904253
17958    904254
18210    984012
Name: stop_id, Length: 918, dtype: object

In [22]:
stops = process_feeds()

Processing feed: /home/data/transit_feed_data/mta_feeds/google_transit_bronx
Processing feed: /home/data/transit_feed_data/mta_feeds/google_transit_brooklyn
Processing feed: /home/data/transit_feed_data/mta_feeds/google_transit_queens
Processing feed: /home/data/transit_feed_data/mta_feeds/google_transit_manhattan
Processing feed: /home/data/transit_feed_data/mta_feeds/google_transit_staten_island
Processing feed: /home/data/transit_feed_data/mta_feeds/mta_subway
Processing feed: /home/data/transit_feed_data/mta_feeds/bus_company
Processing feed: /home/data/transit_feed_data/hrt_feeds/hrt
Processing feed: /home/data/transit_feed_data/hrt_feeds/williamsburg


In [78]:
stops.columns

Index(['stop_id', 'route_type', 'stop_name', 'stop_lat', 'stop_lon',
       'feed_name', 'routes_serviced', 'routes_serviced_str', 'GEOID_2010',
       'GEOID_2020', 'geometry', 'city'],
      dtype='object')

In [86]:
stops.groupby("feed_name").agg(count = ("stop_id", "nunique"))

,count
feed_name,
bus_company,3561
google_transit_bronx,1873
google_transit_brooklyn,4655
google_transit_manhattan,1825
google_transit_queens,2113
google_transit_staten_island,1966
hrt,2694
mta_subway,991
williamsburg,301


In [87]:
stops.agg(count = ("stop_id", "nunique"))

,stop_id
count,19003


In [59]:
bx_stops = make_stops("/home/data/transit_feed_data/mta_feeds/google_transit_bronx")

In [111]:
stops.query("stop_id == '404028'")

,stop_id,route_type,stop_name,stop_lat,stop_lon,feed_name,routes_serviced,routes_serviced_str,GEOID_2010,GEOID_2020,geometry,city
0,404028,Bus,BROADWAY/W 219 ST,40.871953,-73.913137,google_transit_bronx,"M100, BX7, BX20","M100, BX7, BX20",36061029700,36061029700,POINT (-73.91314 40.87195),NYC
16932,404028,Bus,BROADWAY/W 219 ST,40.871770,-73.913094,bus_company,BXM1,BXM1,36061029900,36061029900,POINT (-73.91309 40.87177),NYC


In [67]:
stops.stop_id

0         404028
1         404270
2         403485
3         405435
4         405436
          ...   
19974    2576874
19975    2576873
19976    2576885
19977    2576884
19978    2576883
Name: stop_id, Length: 19979, dtype: object

In [72]:
duplicated_stops.head(50)

635    100025
637    100027
641    100033
644    100039
648    100045
649    100047
650    100048
658    100058
660    100060
670    100071
672    100077
678    100088
679    100089
682    100098
684    100100
685    100101
687    100103
690    100109
694    100114
700    100131
721    100166
732    100183
785    100294
787    100296
789    100298
798    100322
801    100325
803    100328
805    100330
839    100459
841    100461
842    100462
843    100463
844    100464
846    100466
848    100469
850    100471
851    100472
852    100473
853    100474
856    100483
913    100588
914    100589
915    100590
916    100591
917    100592
918    100594
919    100595
923    100609
924    100610
Name: stop_id, dtype: object